# Experiment template. 
## Note: 
This is a template for expariment, which can be run in /run_*ipynb file

In this experiment we use Unet NN structure, general idea is described https://arxiv.org/pdf/1505.04597.pdf .
Model for the current experiment can be found in code/model/models/UnetDeepSoftmax. 

The prediction is a 3D Tensor after softmax, contained 2 masks for lesions and nonlesion probability distributions.  

A weighted loss is used to tackle unbalance in number of lesion/nonlesion. Take a look at parameters lesion_weights, nonlesion_weights (1, 0.1 for this experiment)

For ground truth is presented as ellipses built on line segments annotation.

In [ ]:
# description = {here could be a short description of the experiment}

# specific name of the experiment
eval_name = 'unet_weightedLoss_gtEllip'

if eval_name is None:
    with open(path_to_dir+'eval_name.txt') as data_file:    
        eval_name = json.load(data_file)
print "eval_name is", eval_name 

## Task: segmentation of the image
***
### Content:
* [Settings and experiment parameters](#sep)
* [Load Data](#ld)
* [Training and visualizing results](#lav)
* [Conclusions](#c)
***

#### Togle ON/OFF the raw code

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
CODE WAS HIDDEN. TO TOGGLE ON/OFF THE RAW CODE, CLICK
<a href="javascript:code_toggle()">here</a>.''')

---
<a name="sep"/>
# Settings and experiment parameters
</a>

In [ ]:
%env THEANO_FLAGS="device=gpu1"

### Check theano ####
import theano

### Global variables and paths 
* $\textbf{Add the main directory '.../code' to   sys.path}$. 

The following directory was added:

In [ ]:
#### Add the main dir to sys ####

import os, sys

parentdir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
print parentdir

sys.path.insert(0, parentdir)

In [ ]:
%load_ext autoreload
%autoreload 2

* $\textbf{Put your certain values or None}$ (then use run.ipynb to set them outside).

Note: In case you put None, files with parameters should be in the same directory with this .ipynb file 
(else change path_to_dir by what ever you want)


In [ ]:
TRAIN_NN = True

N_FILTERS = 32
BATCH_SIZE = 50
N_EPOCHS = 10000
PATCH_SIZE = 290

In [ ]:
# Read global params from files
import json

print "N_FILTERS = ", N_FILTERS
print "BATCH_SIZE = ", BATCH_SIZE
print "N_EPOCHS = ", N_EPOCHS
print "PATCH SIZE = ", PATCH_SIZE

In [ ]:
# txt files with paths to segmentation image and input image
from config import txt_train, txt_valid, txt_test

from config import results_path
# path to save the results for THIS experiment
results_eval_path = results_path + eval_name + "/"

print 'txt train path:', txt_train
print 'txt valid path:', txt_valid
print 'txt test path:', txt_test

print 'results_path', results_path

In [ ]:
##### IMPORTS ####
import matplotlib
matplotlib.use('Pdf')

import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

---
<a name='ld'/>
# Load Data
</a>

* Load and visualize data

In [ ]:
from PIL import Image

def load_data(txt, augment=True, masked_data=False):
    ':@return: lists of paths'
    data_imgs = []
    target_imgs = []
    if masked_data:
        masked_data_imgs = []
    else: masked_data_imgs = None
        
    with open(txt, 'r') as fin:
        lines = fin.read().splitlines()
    for line in lines:
        data_imgs.append(line.split(' ')[1]) 
        target_imgs.append(line.split(' ')[0])
        if masked_data:
            masked_data_imgs.append(line.split(' ')[2])
    
    assert(len(data_imgs) == len(target_imgs))
    data_imgs.sort()
    target_imgs.sort()
    if masked_data:
        assert(len(data_imgs) == len(masked_data_imgs))
        masked_data_imgs.sort()
    
    return data_imgs, target_imgs, masked_data_imgs

In [ ]:
X_train, Y_train, X_mask_train = load_data(txt_train, masked_data=True)
X_valid, Y_valid, X_mask_valid = load_data(txt_valid, masked_data=True)
X_test, Y_test, X_mask_test = load_data(txt_test, masked_data=True)

In [ ]:
print 'Train set: data, target, masks', len(X_train), len(Y_train), len(X_mask_train)
print 'Valid set: data, target, masks', len(X_valid), len(Y_valid), len(X_mask_valid)
print 'Test set: data, target, masks', len(X_test), len(Y_test), len(X_mask_test)

---
<a name='lav'/>
# Training and Visualizing
</a>

In [ ]:
from utils.generator import random_crop_generator, threaded_generator
from utils.generator import batch_generator_from_paths as batch_generator

* $ \textbf{ Characteristics of input images and the input layer}$

In [ ]:
#### PREPARE DATA FOR NN ####

import theano.tensor as T
import lasagne
from lasagne.layers import InputLayer

X_data = plt.imread(X_train[0])
print 'Image shape:', X_data.shape

nmb_channels, inp_shape = X_data.shape[2], X_data.shape[:2]
print 'Image size:', inp_shape
print "Number of channels:", nmb_channels

X_inp = T.tensor4('X_inp')
X_layer = InputLayer([BATCH_SIZE, nmb_channels, PATCH_SIZE, \
                      PATCH_SIZE], input_var=X_inp,name='input')

print "Input layer shape:", X_layer.output_shape

# NN

In [ ]:
from models.unetDeepSoftmax import uNet

* $ \textbf{ Characteristics of NN}$

In [ ]:
#### LOAD NN ####
cnn = uNet(X_layer,n_filters=N_FILTERS,nmb_out_classes=2,\
           pad='valid')

In [ ]:
### WEIGHTS sanity check ###
total_weights = int(T.sum([T.prod(w.shape) for w in cnn.weights]).eval())
print "Total weights:", total_weights

* $ \textbf{Objective loss, updates, train and eval fuctions}$

In [ ]:
#### GROUND TRUTH and CLASS_WEIGHTS theano vectors ####
Y_gt = T.ivector('Target Y integer')
weights = T.vector('Loss weights')

In [ ]:
#### LOSS ####
ce,reg_l2, acc, max_0_pred, max_1_pred = cnn.get_loss_components(Y_gt, weights)
loss = ce

In [ ]:
updates = lasagne.updates.adam(loss,cnn.weights,learning_rate=1e-4,\
                               beta1=0.8,beta2=0.999,epsilon=1e-08)

In [ ]:
train_func = theano.function([X_inp,Y_gt,weights], [ce,reg_l2, acc, max_0_pred, max_1_pred], \
                             updates=updates, on_unused_input='warn')

In [ ]:
#### FOR DEBUG ####
# from theano.compile.debugmode import DebugMode
# theano.config.exception_verbosity='high'
# T.cmp_sloppy=1
# train_func = theano.function([X_inp,Y_gt], [loss, acc_train], updates=updates, mode=DebugMode(check_isfinite=0))

In [ ]:
eval_func = theano.function([X_inp, Y_gt, weights], [ce,reg_l2,acc,max_0_pred, max_1_pred], on_unused_input='warn')

In [ ]:
print cnn.outlayer_seg.output_shape

In [ ]:
# seg_output = lasagne.layers.get_output(cnn.outlayer_seg, X_inp)
# seg_output = seg_output.argmax(axis=1)
# get_segmentation = theano.function([X_inp], seg_output)

In [ ]:
# import scipy.misc

# layers = ['pool4','convde_4_2']

# layer_output = {}
# get_fmap = {}

# for layer_name in layers:
#     layer_output[layer_name] = lasagne.layers.get_output(cnn.net[layer_name], X_inp)
#     get_fmap[layer_name] = theano.function([X_inp], layer_output[layer_name])

# def save_feature_maps(layer_name, data, target, path_to_save, epoch=0):
#     fmaps_batch = get_fmap[layer_name](data)
#     fmaps = fmaps_batch[0] 
#     scipy.misc.imsave(os.path.join(path_to_save, 'data_epoch%d.jpg'\
#                                    %epoch),data[0].transpose(1,2,0))
#     scipy.misc.imsave(os.path.join(path_to_save, 'target_epoch%d.jpg'\
#                                    %epoch),target[0,0,:,:])
#     for i,fmap in enumerate(fmaps):
#         scipy.misc.imsave(os.path.join(path_to_save, 'fmap_epoch%d_'\
#                                 %epoch+layer_name+'_%d.jpg'%i), fmap)
        


In [ ]:
#### Note: Set weights for classes here ####
lesion_weight = 1.
nonlesion_weight = 0.1

### Visualization

In [ ]:
from utils.predict_full_img import predict_full_img, process_img_for_plot, plot_dsp

In [ ]:
from visualizers.plot_predictions import plot_predictions

In [ ]:
def plot_predictions(pred_fn, generator, BATCH_SIZE, path_to_save, \
                     n_images=10,info = False, info_threshold=0.005):
    cnt_images = 0            
    for data, seg, masked_data in generator:
        # shape [batch_size, nmb_classes=2, output_shape. output_shape]
        pred = pred_fn(data)
        
        # if None transform to list for iteration
        if masked_data is None:
            masked_data=[None]*len(data)
            
        for i, (d, s, md, p) in enumerate(zip(data, seg, masked_data, pred)):
            # use only images with informational content (nmb lesion pixels) more than threshold
            if info:
                info_percent = np.sum(s > 0)*1./ np.size(s.ravel())
                if info_percent < info_threshold:
                    pass                 
                else: 
                    if md is not None:
                        img_edged, gt_edged, plot_img_gt, plot_img_pred, diff_gt_pred = \
                        process_img_for_plot(d.transpose(1,2,0), s[0], p[0], 0.3, md.transpose(1,2,0))
                    else:
                        img_edged, gt_edged, plot_img_gt, plot_img_pred, diff_gt_pred = \
                        process_img_for_plot(d.transpose(1,2,0), s[0], p[0], rate=0.3)
                        
                    plot_dsp(p[0], img_edged, gt_edged, plot_img_gt, plot_img_pred, diff_gt_pred, \
                         path_to_save=path_to_save+"{}.png".format(i))
                    
                    cnt_images+=1
                    
            # no informational content
            else:
                if md is not None:
                    img_edged, gt_edged, plot_img_gt, plot_img_pred, diff_gt_pred = \
                    process_img_for_plot(d.transpose(1,2,0), s[0], p[0], 0.3, md.transpose(1,2,0))
                else:
                    img_edged, gt_edged, plot_img_gt, plot_img_pred, diff_gt_pred = \
                    process_img_for_plot(d.transpose(1,2,0), s[0], p[0], rate=0.3)
                
                plot_dsp(p[0], img_edged, gt_edged, plot_img_gt, plot_img_pred, diff_gt_pred, \
                     path_to_save=path_to_save+"{}.png".format(i))
                
                cnt_images+=1
        if cnt_images > n_images:
            break

* $ \textbf{Predict full image}$

* $ \textbf{Folders with results}$

In [ ]:
import os
from os.path import join

path_metrics_plot = './metrics_plot'
path_pred_small_img = './pred_small_img'
path_pred_full_img = './pred_full_img'
path_snapshots = './snapshots'
path_fmaps = './fmaps'

for folder in [path_metrics_plot, path_pred_small_img, path_pred_full_img, path_snapshots,\
              path_fmaps]:
    if not os.path.exists(folder):
        os.makedirs(folder)

* $ \textbf{Data Generators}$

In [ ]:
RS_TRAIN = 10 # random seed

batch_gen = batch_generator(X_train, Y_train, BATCH_SIZE, \
                            random_seed=RS_TRAIN, masked_data_paths=X_mask_train)

train_generator = random_crop_generator(batch_gen, 
                                        info=True, 
                                        crop_size=cnn.input_img_shape, 
                                        target_crop_size=cnn.pred_img_shape, 
                                        random_seed=RS_TRAIN)

train_generator = threaded_generator(train_generator, num_cached=5)

In [ ]:
RS_VALID = 20 # random seed

batch_gen = batch_generator(X_valid, Y_valid, BATCH_SIZE, \
                            random_seed=RS_VALID, masked_data_paths= X_mask_valid)

validation_generator = random_crop_generator(batch_gen, 
                                             info=True, 
                                             crop_size=cnn.input_img_shape, 
                                             target_crop_size=cnn.pred_img_shape,
                                             random_seed=RS_VALID)

validation_generator = threaded_generator(validation_generator, num_cached=5)

In [ ]:
valid_fullimg_gen = batch_generator(X_valid, Y_valid, BATCH_SIZE, \
                                random_seed=RS_VALID, info=True, info_threshold = 0.01,\
                                   masked_data_paths = X_mask_valid)

### Visualizer

In [ ]:
from visualizers.metrics_visualizer import MetricsVisualizer

* $ \textbf{Train NN}$

In [ ]:
NMB_SAMPLES = 20

In [ ]:
from utils.persistence import *
import time, datetime, pytz
from visualizers.metrics import Metrics
from datetime import datetime as dt

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.metrics import roc_curve, recall_score, precision_score


metrics = Metrics()
acc_metrics = Metrics()

def cur_time_fn():
    return dt.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d %H:%M")

def train_nn (N_EPOCHS):
    
    with open(logs_path, "a+") as logs:
                cur_time = cur_time_fn()
                logs.write("\n\nEval {}, \nCurrent time {} \n"\
               .format(eval_name, cur_time))
                
    losses = [] 
    losses_val = []
    accs = []
    accs_val = []
    epoch = 1

    t_start = time.time()
    
    for i in range(N_EPOCHS):
        
        print "\nepoch", epoch
        n_batches = 0
        t0 = time.time()
        
        ## generate train batch
        data, target, _ = train_generator.next()
        target_flat = target.flatten()
        ## weights for target elements
        weights_target = np.where(target_flat<0.5, nonlesion_weight, lesion_weight).astype(np.float32)
        ## compute loss for train batch 
        ce_i, reg_i, acc_i, max_pred_i, min_pred_i = \
                train_func(data.astype(np.float32),target_flat,weights_target)   
        
        print "sum, max_pred, min_pred", max_pred_i[0]+min_pred_i[1], max_pred_i, min_pred_i
        ## sanity check of metrics
        print 'Loss sanity check: ce_i %.3f, reg_i %.3f, acc_i %.3f'%(ce_i,reg_i,acc_i)    
            
        ## approximate class balance info
        if epoch==1:
            nmb_nonles, nmb_les = np.sum(target==0), np.sum(target==1)
            class_balance_info = "\nClass Balance for 1 epoch: rate {:.2f}, nonlesions {}, lesions {}\n"\
                                    .format(nmb_les*1./nmb_nonles, nmb_nonles, nmb_les)            
            print class_balance_info
            with open(logs_path, "a+") as logs:
                logs.write(class_balance_info)

        loss_i = ce_i
        metrics["train loss"][epoch] = loss_i
#         metrics["train reg"][epoch] = reg_i
        acc_metrics["train acc"][epoch] = acc_i
        losses.append(loss_i) 
        accs.append(acc_i)
            
        ## save metrics for train and validation
        if epoch%10==0:
            
            ## validation batch
            data, target, masked_data = validation_generator.next()
            target_flat = target.flatten()
            weights_target = np.where(target_flat<0.5, nonlesion_weight, lesion_weight).astype(np.float32)
            ce_i, reg_i, acc_i, max_pred_i, min_pred_i = \
                    eval_func(data.astype(np.float32), target_flat, weights_target)
            loss_i = ce_i
            
            metrics["test loss"][epoch] = loss_i
#             metrics["test reg"][epoch] = reg_i 
            acc_metrics["test acc"][epoch] = acc_i
            losses_val.append(loss_i)
            accs_val.append(acc_i)

            print('#'*30)
            mean_train_loss = np.mean(losses[-10:])
            print('%d \t| ce\t| %.3f\t| %.3f'%(epoch, mean_train_loss, losses_val[-1]))
#             print('%d \t| auc| %.3f\t| %.3f'%(epoch, np.mean(metrics["train auc"][-10:]), auc_i))
            print('#'*30)
            
            with open(logs_path, "a+") as logs:
                cur_time = cur_time_fn()
                logs.write("""\nEpoch %d\t| Train Loss %.3f\t| Val Loss %.3f\t| Train ACC %.3f\t| Val ACC %.3f\t |Cur time: %s"""\
               %(epoch, mean_train_loss, losses_val[-1], np.mean(accs[-10:]), acc_i, cur_time))
        
        if epoch%200==0:
            ## plot and save metrics evaluation
            fig = plt.figure(figsize=[15,5])
            name = eval_name+"_metrics{}epoch.png".format(epoch)
            path_save_plot = join(path_metrics_plot,name)
            metrics.plot()
            fig.savefig(path_save_plot)
            fig = plt.figure(figsize=[15,5])
            name = eval_name+"_ACCmetrics{}epoch.png".format(epoch)
            path_save_plot = join(path_metrics_plot,name)
            acc_metrics.plot()
            fig.savefig(path_save_plot)

            ## weights snapshort
            name = eval_name + '_weights{}epoch'.format(epoch) + '.pickle'
            file_weights_path = join(path_snapshots, name)
            save(cnn.outlayer_for_loss, file_weights_path)
            
            ## visualize small predictions
            name = eval_name+'_pred_small_img{}epoch'.format(epoch)
            plot_predictions(cnn.predict,validation_generator,BATCH_SIZE,info=True, n_images=NMB_SAMPLES,
                              path_to_save=join(path_pred_small_img, name))
        
        ## predict full validation image
        if epoch%400==0:
            with open(logs_path, "a+") as logs:
                logs.write("""\nPredicting full image...\t| Cur time: %s"""%(cur_time_fn()))
                
            data, target, masked_data = valid_fullimg_gen.next()
            
            print cur_time_fn(), 'Predicting full image...'
            pred = predict_full_image(data, model=cnn) 
            print cur_time_fn(), 'Done. Process for plot...'
            
            with open(logs_path, "a+") as logs:
                logs.write("""\nDone.\t| Cur time: %s\t| Plotting..."""%(cur_time_fn()))

            # if None transform to list for iteration
            if masked_data is None:
                masked_data=[None]*len(data)
            
            for i, (d, s, md, p) in enumerate(zip(data, target, masked_data, pred)):
                p_les = p[1]
                p_nles = p[0]
                if md is not None:
                    img_edged, gt_edged, plot_img_gt, plot_img_pred, diff_gt_pred = \
                    process_img_for_plot(d.transpose(1,2,0), s[0], p_les, 0.3, md.transpose(1,2,0))
                else:
                    img_edged, gt_edged, plot_img_gt, plot_img_pred, diff_gt_pred = \
                    process_img_for_plot(d.transpose(1,2,0), s[0], p_les, rate=0.3)
        
                np.save(join(path_pred_full_img,'pred{}epoch{}'.format(epoch,i)), p_les)
                np.save(join(path_pred_full_img,'pred_oppos{}epoch{}'.format(epoch,i)), p_nles)
                np.save(join(path_pred_full_img,'img_edged{}epoch{}'.format(epoch,i)), img_edged)
                np.save(join(path_pred_full_img,'gt_edged{}epoch{}'.format(epoch,i)), gt_edged)
                
                name = 'pred_full_img{}epoch{}.png'.format(epoch,i)
                plot_dsp(p_les, img_edged, gt_edged, plot_img_gt, plot_img_pred, diff_gt_pred, \
                         pred_map2 = p_nles, bin_pred = p.argmax(axis=0), path_to_save=join(path_pred_full_img,name))
                print i, 'saved and ploted'                
                if i > NMB_SAMPLES: break
            
            with open(logs_path, "a+") as logs:
                logs.write("""\nDone.\t| Cur time: %s"""%(cur_time_fn()))
            
            np.save(join(path_snapshots,'losses.npy'), losses)
            np.save(join(path_snapshots, 'losses_val.npy'), losses_val)
            np.save(join(path_snapshots,'acc.npy'), accs)
            np.save(join(path_snapshots, 'acc_val.npy'), accs_val)
                    
        
#         # save feature maps
#         if epoch%1000==0:  
#             for l in layers:
#                 try:
#                     save_feature_maps(l, data, target, './fmaps', epoch)
#                 except: pass


        epoch+=1
        print 'time for epoch: %.2f mins'%((time.time() - t0)/60.)
    print 'Overall time: %.2f mins'%((time.time() - t_start)/60.)

In [ ]:
#### TRAIN NN ####
logs_path = './logs.txt'

print "N_EPOCHS = ", N_EPOCHS

# nn_weights = './snapshots/debug_weights400epoch.pickle'
# print 'path to nn weights', nn_weights

try:
#     print "\nLoading weights..."
    a = load(cnn.outlayer_for_loss, nn_weights)
except:
    print "problem with weights loading, nn is being trained\n"
else: 
    print "weights are loaded\n"
if TRAIN_NN:
    train_nn(N_EPOCHS) 

## Apply nn

In [ ]:
from utils.persistence import *
import time, datetime, pytz
from visualizers.metrics import Metrics
from datetime import datetime as dt

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.metrics import roc_curve, recall_score, precision_score


metrics = Metrics()

def cur_time_fn():
    return dt.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d %H:%M")

def apply_nn(N):
    
    with open(logs_path, "a+") as logs:
                cur_time = cur_time_fn()
                logs.write("\n\nVALIDATION. Eval {}, \nCurrent time {} \n"\
               .format(eval_name, cur_time))
    epoch = 1
    losses_val = []
    accs_val = []
    t_start = time.time()
    
    for i in range(N):
        
        print "\nepoch", epoch
        n_batches = 0
       
        ## save metrics for validation
        
        ## validation batch
        data, target = validation_generator.next()
        target_flat = target.flatten()
        weights_target = np.where(target_flat<0.5, nonlesion_weight, lesion_weight).astype(np.float32)
        ce_i, reg_i, acc_i, max_pred_i, min_pred_i = \
                eval_func(data.astype(np.float32), target_flat, weights_target)
        loss_i = ce_i

        metrics["test loss"][epoch] = loss_i
#             metrics["test reg"][epoch] = reg_i 
        acc_metrics["test acc"][epoch] = acc_i
        losses_val.append(loss_i)
        accs_val.append(acc_i)

        print('#'*30)
        print('%d \t| ce |  %.3f'%(epoch, losses_val[-1]))
        print('#'*30)

        with open(logs_path, "a+") as logs:
            cur_time = cur_time_fn()
            logs.write("""\nIteration %d\t|| Val Loss %.3f\t | Cur time: %s"""\
           %(epoch, losses_val[-1], cur_time))
        
        ### PLOTTING
        ## visualize small predictions
        print cur_time_fn(), 'Plot_predictions..'
        plot_predictions(get_segmentation,validation_generator,BATCH_SIZE,info=True, n_images=NMB_SAMPLES,
                          path_to_save='predictions/predictions_iteration{}.png'.format(epoch))
        print cur_time_fn(), "Done."
        
        ##FULL
        ## predict full validation image
        data, target = valid_fullimg_gen.next()

        print cur_time_fn(), 'Predicting full image...'
        pred = predict_full_image(data, model=cnn) 
        plt.imshow(pred[0,0])
        plt.show()
        print cur_time_fn(), 'Done. Process for plot...'

        for i, (d,gt,p) in enumerate(zip(data, target, pred)):
            img_edged, gt_edged, plot_img_gt, plot_img_pred, diff_gt_pred = \
                process_img_for_plot(d.transpose(1,2,0), gt[0], p[0], rate=0.3)
            print i, 'process_img_for_plot'
            plot_dsp(p[0], img_edged, gt_edged, plot_img_gt, plot_img_pred, diff_gt_pred, \
                     path_to_save='pred_full_img/pred_full_img_iteration{}_{}.png'.format(epoch,i), show=True)
            if i > NMB_SAMPLES: break
        
#         # save feature maps
#         if epoch%1000==0:  
#             for l in layers:
#                 try:
#                     save_feature_maps(l, data, target, './fmaps', epoch)
#                 except: pass


        epoch+=1
        print 'time for epoch: %.2f mins'%((time.time() - t0)/60.)
    print 'Overall time: %.2f mins'%((time.time() - t_start)/60.)

In [ ]:
#### TRAIN NN ####
logs_path = './logs.txt'

N = 10

nn_weights = 'snapshots/'+ 'unet_adam_lr1e-2_weights8000epoch.pickle'
# print 'path to nn weights', nn_weights

try:
#     print "\nLoading weights..."
    a = load(cnn.outlayer_for_loss, nn_weights)
except:
    print "problem with weights loading, nn is being trained\n"
else: 
    print "weights are loaded\n"
apply_nn(N) 

* $ \textbf{Save weights}$ 

In [ ]:
from utils.persistence import *

if not os.path.exists(results_eval_path):
    os.makedirs(results_eval_path)
    
file_path = results_eval_path + eval_name + '_weights' + '.pickle'
save(cnn.outlayer_for_loss, file_path)

In [ ]:
#### TEST SAVING ####
try:
    a = load(cnn.outlayer_for_loss, nn_weights)
except:
    print "The problem occured. Weights were not saved"
else: print 'Weights were successfully saved to the file: ', file_path

* $ \textbf{Visualizations}$ 

---
<a name='c'/>
# Conclusions
</a>